# RAG Chatbot using LangChain ConversationalRetrievalChain with Zephyr 7b gptq model

In [1]:
!pip install langchain chainlit pyngrok google-search-results transformers==4.37.2 pypdf sentence-transformers chromadb auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.

In [2]:
import langchain
import chainlit
import pyngrok
import transformers
import pypdf
import chromadb
import auto_gptq


print(langchain.__version__)
print(chainlit.__version__)
print(pyngrok.__version__)
print(transformers.__version__)
print(pypdf.__version__)
print(chromadb.__version__)
print(auto_gptq.__version__)

0.1.16
1.0.504
7.1.6
4.37.2
4.2.0
0.4.24
0.7.1


In [3]:
from langchain.document_loaders import PyPDFLoader
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, TextStreamer
from auto_gptq import AutoGPTQForCausalLM
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain
import os
import re

In [4]:
if not os.path.exists("Vector_db"):
  os.makedirs("Vector_db")

loader = PyPDFLoader("/content/e_budget_speech_2022-23.pdf")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
Vector_db_01 = Chroma.from_documents(docs, embeddings, collection_name="23budget", persist_directory="Vector_db")
Vector_db_01.persist()
Vector_db_01 = None

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
Vector_db_01 = Chroma(persist_directory="Vector_db", collection_name="23budget", embedding_function=embeddings)
Vector_db_01.get()
Vector_db_01.similarity_search("Is there any economic support for start up company in Hong Kong 2023 budget?")

[Document(page_content='Enhancing Economic Resilience and Enriching Industrial Development  \n \n \n23 55. Over the past few years, apart from the $2  billion \nInnovation and Technology Venture Fund, we have also set up the \nCorporate Venture Fund and the Cyberport Macro Fund through the \nHKSTPC and the Cyberport respectively.  Each of these funds has its own specific key areas of investment, which has significantly broadened the fundraising channel for start -ups in Hong Kong.  \n56. Nevertheless, as the funds mentioned above are mainly \ntargeted at early- stage start -ups, they may not be suitable for expanding \nenterprises with considerable scale.  Some such type of enterprises \noften have huge development potential.  Indeed, we are not short of \nexamples of success in nurturing these enterprises in Hong Kong.  \n57. I announced in my Budget two years ago the setting up of \nan investment portfolio named the Hong Kong Growth Portfolio using part of the Future Fund for investm

In [6]:
def load_llm_zephyr_7B():

  model_name = "TheBloke/zephyr-7B-beta-GPTQ"
  model_basename = "model"
  revision="gptq-4bit-64g-actorder_True"
  DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

  tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

  model = AutoGPTQForCausalLM.from_quantized(
      model_name,
      revision=revision,
      model_basename=model_basename,
      use_safetensors=True,
      trust_remote_code=True,
      inject_fused_attention=False,
      device=DEVICE,
      quantize_config=None,)

  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

  text_pipeline = pipeline(
      "text-generation",
      model=model,
      tokenizer=tokenizer,
      do_sample=False,
      # temperature=0,
      max_new_tokens=1024,
      streamer=streamer,
  )

  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})

  return llm

def set_prompt():

  template = """You are a helpful, respectful and honest assistant.
          Answer the question in your own words as truthfully as possible from the context given to you.
          If you do not know the answer to the question, simply respond with "I don't know. Can you ask another question".
          If questions are asked where there is no relevant context available, simply respond with "I don't know. Please ask a question relevant to the documents"

          Context: {context}

          Human: {question}
          Assistant:"""

  zephyr_prompt = PromptTemplate(input_variables=["context", "question"], template=template)

  return zephyr_prompt

def set_prompt_with_chat_history():

  template = """You are a helpful, respectful and honest assistant.
          Answer the question in your own words as truthfully as possible from the context given to you.
          If you do not know the answer to the question, simply respond with "I don't know. Can you ask another question".
          If questions are asked where there is no relevant context available, simply respond with "I don't know. Please ask a question relevant to the documents"

          Context: {context}


          Chat history: {chat_history}

          Human: {question}
          Assistant:"""

  zephyr_prompt = PromptTemplate( input_variables=["context", "chat_history", "question"], template=template)

  return zephyr_prompt

def set_qa_chain():

  memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", return_messages=True)

  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  Vector_db_01 = Chroma(persist_directory="Vector_db", collection_name="23budget", embedding_function=embeddings)
  Vector_db_01.get()

  llm = load_llm_zephyr_7B()
  qa_chain_prompt = set_prompt_with_chat_history()


  qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=Vector_db_01.as_retriever(),
                            memory=memory, combine_docs_chain_kwargs={'prompt': qa_chain_prompt})

  return qa_chain

In [7]:
qa_bot = set_qa_chain()

qa_bot

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

quantize_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.29G [00:00<?, ?B/s]

INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.
The model 'MistralGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpFo

ConversationalRetrievalChain(memory=ConversationBufferMemory(output_key='answer', input_key='question', return_messages=True, memory_key='chat_history'), combine_docs_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['chat_history', 'context', 'question'], template='You are a helpful, respectful and honest assistant.\n          Answer the question in your own words as truthfully as possible from the context given to you.\n          If you do not know the answer to the question, simply respond with "I don\'t know. Can you ask another question".\n          If questions are asked where there is no relevant context available, simply respond with "I don\'t know. Please ask a question relevant to the documents"\n\n          Context: {context}\n\n\n          Chat history: {chat_history}\n\n          Human: {question}\n          Assistant:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7859ac493eb

In [8]:
query="Is there any economic support for start up company in Hong Kong 2023 budget?"
res = qa_bot.invoke({"question": query})

Yes, the Hong Kong Government has announced the setting up of a new investment fund, namely the Strategic Tech Fund, to nurture enterprises that are relatively more mature and have good potential for contribution to the economy. The Government will increase the funding allocated to the Hong Kong Growth Portfolio under the Future Fund by $10 billion, of which $5 billion will be used to set up the Strategic Tech Fund. The Hong Kong Science and Technology Parks Corporation (HKSTPC) and the Cyberport will be invited to identify technology enterprises which are of strategic value to Hong Kong as well as investment opportunities conducive to enriching the I&T ecosystem. This initiative aims to consolidate Hong Kong’s status as a financial, commercial, and I&T centre and raise its productivity and competitiveness in the long run.


In [9]:
answer = re.findall(r"Assistant: (.*)", res["answer"])[-1].strip()
answer

'Yes, the Hong Kong Government has announced the setting up of a new investment fund, namely the Strategic Tech Fund, to nurture enterprises that are relatively more mature and have good potential for contribution to the economy. The Government will increase the funding allocated to the Hong Kong Growth Portfolio under the Future Fund by $10 billion, of which $5 billion will be used to set up the Strategic Tech Fund. The Hong Kong Science and Technology Parks Corporation (HKSTPC) and the Cyberport will be invited to identify technology enterprises which are of strategic value to Hong Kong as well as investment opportunities conducive to enriching the I&T ecosystem. This initiative aims to consolidate Hong Kong’s status as a financial, commercial, and I&T centre and raise its productivity and competitiveness in the long run.'

In [10]:
query="Tell me about the $5 billion that you have mentioned."
res = qa_bot.invoke({"question": query})

How will the $5 billion allocated to the Strategic Tech Fund be used to support mature enterprises in Hong Kong?
The $5 billion allocated to the Strategic Tech Fund will be used to support mature enterprises in Hong Kong that have significant development potential. The Hong Kong Science and Technology Parks Corporation (HKSTPC) and the Cyberport will be invited to identify technology enterprises which are of strategic value to Hong Kong as well as investment opportunities conducive to enriching the I&T ecosystem. This initiative aims to consolidate Hong Kong’s status as a financial, commercial, and I&T centre and raise its productivity and competitiveness in the long run.


In [11]:
answer = re.findall(r"Assistant: (.*)", res["answer"])[-1].strip()
answer

'The $5 billion allocated to the Strategic Tech Fund will be used to support mature enterprises in Hong Kong that have significant development potential. The Hong Kong Science and Technology Parks Corporation (HKSTPC) and the Cyberport will be invited to identify technology enterprises which are of strategic value to Hong Kong as well as investment opportunities conducive to enriching the I&T ecosystem. This initiative aims to consolidate Hong Kong’s status as a financial, commercial, and I&T centre and raise its productivity and competitiveness in the long run.'